# No Olicoclonal Bands

In [12]:
import numpy as np
import pandas as pd
import cirq
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Example synthetic dataset
data = pd.DataFrame({
    'CSF_IgG': np.random.uniform(2, 10, 50),  # Random values for demonstration
    'CSF_Albumin': np.random.uniform(10, 30, 50),
    'Serum_IgG': np.random.uniform(5, 15, 50),
    'Serum_Albumin': np.random.uniform(40, 50, 50),
    'has_ocb': np.random.randint(0, 2, 50)  # Random binary labels
})

# Calculate IgG Index
data['IgG_Index'] = (data['CSF_IgG'] / data['Serum_IgG']) / (data['CSF_Albumin'] / data['Serum_Albumin'])

# Prepare features and labels
X = data[['IgG_Index']].values
y = data['has_ocb'].values

# Oliclonal Bands Present

In [15]:
# Creating a synthetic dataset with a specific case having oligoclonal bands
data = pd.DataFrame({
    'CSF_IgG': np.random.uniform(2, 10, 49).tolist() + [15.0],  # Add a high value to ensure OCB presence
    'CSF_Albumin': np.random.uniform(10, 30, 49).tolist() + [25.0],
    'Serum_IgG': np.random.uniform(5, 15, 49).tolist() + [12.0],
    'Serum_Albumin': np.random.uniform(40, 50, 49).tolist() + [45.0],
    'has_ocb': np.random.randint(0, 2, 49).tolist() + [1]  # Add a case with OCB
})

# Calculate IgG Index
data['IgG_Index'] = (data['CSF_IgG'] / data['Serum_IgG']) / (data['CSF_Albumin'] / data['Serum_Albumin'])

# Prepare features and labels
X = data[['IgG_Index']].values
y = data['has_ocb'].values

In [16]:
# Quantum feature extraction
def create_quantum_circuit(value):
    qubits = cirq.LineQubit.range(1)
    circuit = cirq.Circuit()
    angle = 2 * np.pi * value
    circuit.append(cirq.ry(angle)(qubits[0]))
    circuit.append(cirq.measure(qubits[0], key='result'))
    return circuit

def simulate_circuit(circuit):
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1000)
    counts = result.histogram(key='result')
    return counts

def extract_quantum_features(data):
    features = []
    for value in data:
        circuit = create_quantum_circuit(value)
        counts = simulate_circuit(circuit)
        feature_value = np.mean(list(counts.values()))
        features.append(feature_value)
    return np.array(features).reshape(-1, 1)

# Extract quantum features from data
quantum_features = extract_quantum_features(X.flatten())

In [17]:
# Split data ensuring both classes are represented
X_train, X_test, y_train, y_test = train_test_split(quantum_features, y, test_size=0.3, random_state=42)

# Train a classical model using quantum features
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probability of class 1
print("Predicted probabilities:", y_pred_prob)

# Threshold-based decision
threshold = 0.5
y_pred = (y_pred_prob > threshold).astype(int)

print("Predictions:", y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Print a simple message based on the test predictions
if np.any(y_pred == 1):
    print("Yes, there are oligoclonal bands present in the test set.")
else:
    print("No, there are no oligoclonal bands present in the test set.")

Predicted probabilities: [0.53124988 0.53124988 0.53124988 0.53124988 0.53124988 0.53124988
 0.53124988 0.53124988 0.53124988 0.53124988 0.53124988 0.66666569
 0.53124988 0.53124988 0.53124988]
Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Accuracy: 0.5333333333333333
Yes, there are oligoclonal bands present in the test set.
